# Introduce
CacheQM是一种QM LinkList的Wrapper。把原有QM的header节点修改为Cache节点。其特点有：

* 支持直接写Cache，减少延时和功耗
* 通过Cache节点来驱动外围调度器
  * 通过Cache Node的valid信号来驱动外围的调度器，减少调度器对QM时许的要求


# 定义类

导入相关库

In [43]:
import numpy as np
import pandas as pd
import math

## 类定义

In [48]:
class CacheQM:
    def __init__(self, qnum_l1, qnum_l2, qnum_l3, nnum):
        """
        Arguments:
        * qnum: {int} ---- 队列个数
        * nnum: {int} ---- 节点个数
        """
        self.qnum_l1 = qnum_l1
        self.qnum_l2 = qnum_l2
        self.qnum_l3 = qnum_l3
        self.qnum = qnum_l1 + qnum_l2 + qnum_l3
        self.nnum = nnum

## 寄存器资源评估函数

In [49]:
def reg_num(qm, wr_dly=3, rd_dly=4, enq_wr_link_dly=4, link_mem_depth=0, pre_info_width=0,
           fp4_cache_width=0, fp2_cache_width=0, fp1_cache_width=0):
    """
    Arguments:
    WR_DLY: {int} ---- 写入延时
    """
    
    q_total_num_wid = math.log(qm.qnum, 2)
    cache_width = pre_info_width + q_total_num_wid + 1
    
    regs = pd.DataFrame(columns=['value','description'])
       
    regs.loc['get_addr_ff'] = [qn.nnum*(enq_wr_link_dly+1), '']
    regs.loc['enq_wr_cache'] = [wr_dly+2+enq_wr_link_dly+1, 'WR信号延时']
    regs.loc['enq_req_ff'] = [wr_dly+1+enq_wr_link_dly, 'REQ信号延时']
    regs.loc['enq_qid_ff'] = [q_total_num_wid * (wr_dly+1) + (q_total_num_wid * (enq_wr_link_dly+1)), 'QID延时']
    
    regs.loc['fp2_sub_linkid'] = [qm.qnum_l2,'']
    regs.loc['fp1_sub_linkid'] = [qm.qnum_l3*2, '']
    
    regs.loc['deq_reg_ff'] = [rd_dly+1, '出队req信号延时']
    regs.loc['deq_qid_ff'] = [q_total_num_wid * (rd_dly+1), '出队qid信号延时']
    
    regs.loc['cache_link_waddr_ff'] = [(wr_dly+1) * link_mem_depth, '']
    regs.loc['cache_link_wdat_ff'] = [( link_mem_depth + pre_info_width ) * (wr_dly + 1), '']
    regs.loc['link_raddr_ff'] = [link_mem_depth * (rd_dly + 1), '']
    
    regs.loc['req_deq_equal'] = [1, '']
    regs.loc['deq_drag_vld'] = [(rd_dly+1) * qm.qnum + 1, '']
    regs.loc['fp2_cache_wptr'] = [qm.qnum_l2 * 2, '']
    regs.loc['fp1_cache_wptr'] = [qm.qnum_l3 * 2 * 2, '']
    regs.loc['fp2_pre_alloc_wptr'] = [qm.qnum_l2 * 2, '']
    regs.loc['fp1_pre_alloc_wptr'] = [qm.qnum_l3 * 2 *3,'']
    regs.loc['cah_reload_b2b'] = [(1+link_mem_depth*pre_info_width) * wr_dly, '1bit valid + address']
    regs.loc['cah_reolad_data'] = [cache_width, '']
    
    regs.loc['cache_node'] = [(cache_width*qm.qnum_l1) + (cache_width*qm.qnum_l2*2) + (cache_width*qm.qnum_l3*3), 'Cache节点']
    regs.loc['qlen'] = [math.log(qm.nnum, 2) * qm.qnum, '']
    
    return regs
    

CacheQM.reg_num = reg_num

SyntaxError: invalid syntax (<ipython-input-49-2a35df644fe9>, line 34)

In [50]:
qm1 = CacheQM(16, 4096)

regs = qm1.reg_num(5)

regs

TypeError: __init__() missing 2 required positional arguments: 'qnum_l3' and 'nnum'